In [1]:
import torch
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import tensorflow as tf
import numpy as np 
import pandas as pd

ModuleNotFoundError: No module named 'transformers'

In [ ]:
def predict_subreason(df):
    labels_dict={'siteperformance': 0, 'stock': 1, 'orderandcommunication': 2, 
             'search': 3, 'navigation': 4, 'other': 5, 
             'deliveryinformationandmyaccount': 6, 'sparksandpromotions': 7, 
             'productrange': 8, 'missingcontentorimagery': 9, 'technicalissue': 10,
             'checkoutandlogin': 11, 'covid-19': 12, 'general ': 13}

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(labels_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.load_state_dict(torch.load('BERT_ft_epoch5.model', map_location=torch.device('cpu')))
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    df = df[2:]
    df['Predicted_Sub_Reason']=None
    df['Predicted_value']=None

    labels_dict_new = dict([(value, key) for key, value in labels_dict.items()])
    for i, row in df.iterrows():
      try:
        inputs = tokenizer(row['Comment'], return_tensors="pt")
        inputs.to(device)

        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(**inputs, labels=labels)
        logits=outputs[1]
        logits = logits.detach().cpu().numpy()
        df.at[i,'Predicted_Sub_Reason'] = labels_dict_new[np.argmax(logits)]
        df.at[i,'Predicted_value'] = np.argmax(logits)
      except:
        print('Data type problem',row['Comment'])

    return df

In [19]:
def Majority_Minority(file):
    filename=file+'.xls'
    import json
    import pandas as pd
    import warnings
    warnings.filterwarnings("ignore")
    df_s=pd.read_excel(filename,sheet_name='Dotcom Feedback Comments Report')
    df=predict_subreason(df_s)
    
    from transformers import pipeline
    from pickle import load
    import re
    import nltk
    #nltk.download('stopwords')
    from nltk.corpus import stopwords
    from nltk.stem.porter import PorterStemmer
    
    def Sentiment_Analysis(df_s):
        #print("aaa")
        identifier= pipeline('sentiment-analysis')
        df_s['Output_trans']=''
        df_s['Output_trans']=df_s['Comment'].apply(lambda x: x[:510] if len(x)>510 else x) 
        #%%time
        df_s['output_trans_1'] = df_s['Output_trans'].apply(lambda text : identifier(text))
        df_s['Sentiment']=df_s['output_trans_1'].apply(lambda t: t[0]['label'])
        df_s['polarity_Score']=df_s['output_trans_1'].apply(lambda t: t[0]['score'])
        #print(df.loc[40])
        return df_s

    def PreProcessing(df_p):
        #print("______________")
        Data=df_p[['Comment']]
        Data.dropna(inplace=True)
        data=Data
        data.reset_index(inplace=True)
        #data.drop('index',axis=1,inplace=True)
        X_test=data

       
        corpus = []
        X_test['Issue']=''
        for i in range(len(X_test)):
            #print(i)
            contractions_dict = {
            "didn t": 'did not',
            "don t": 'do not',
            "doesn t": 'does not',
            "won t": 'will not',
            "couldn t":"could not",
            "shouldn t" :"should not",
            "wouldn t" :"would not"
            }
            contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
            #print(i)
            issue = re.sub('[^a-zA-Z]', ' ', str(X_test['Comment'].loc[i]))
            issue = issue.lower()
            def expand_contractions(s, contractions_dict=contractions_dict):
                def replace(match):
                    return contractions_dict[match.group(0)]
                issue=contractions_re.sub(replace, s)
                #print(issue)
                return issue
            #print(issue)
            issue=expand_contractions(issue)

            issue = issue.split()


            ps = PorterStemmer()
            all_stopwords = stopwords.words('english')
            all_stopwords.remove('not')
            all_stopwords.remove('out')
            issue = [ps.stem(word) for word in issue if not word in set(all_stopwords)]
            issue = ' '.join(issue)
            corpus.append(issue)
            X_test['Issue'].iloc[i]=issue
            
        x_test=X_test['Issue']
            
        return x_test,X_test
    
    def Model_Building(x_test,X_test):
        #print("AAAAAAAAAAAAA")
        tv1 = load(open('TF_IDF_Vectorizer.pickle', 'rb'))
        test_tv = tv1.transform(x_test)

        SVCclassifier= load(open('Min_Majority.pickle', 'rb'))

        y_pred_SVM = SVCclassifier.predict(test_tv)
        X_test['Majoity_Minority']=y_pred_SVM
        
        return x_test,X_test
        
    x_test1,X_test1=PreProcessing(df)
    x_test2,X_test2=Model_Building(x_test1,X_test1)
    X_test2=Sentiment_Analysis(X_test2)
    #print(X_test2)
    #print(y_pred_SVM)
    #print(X_test1['Majoity_Minority'].value_counts())
    Final_result=pd.merge(df, X_test2, on="Comment", how = 'left')
    Final_result.drop(['index','Issue','Output_trans','output_trans_1','polarity_Score'],axis=1,inplace=True)
    
    result = Final_result.to_json(orient="records")
    
    return (result)

In [20]:
%%time
Result=Majority_Minority('Day_4')

Wall time: 45.6 s


In [21]:
Result

'[{"Totals":"Totals","M&S Dotcom Hierarchy":null,"Dotcom BU":null,"Comment":null,"Feedback OSAT":4.9,"S_ecid":null,"Majoity_Minority":null,"Sentiment":null},{"Totals":"Totals","M&S Dotcom Hierarchy":"M&S Dotcom","Dotcom BU":null,"Comment":null,"Feedback OSAT":4.9,"S_ecid":null,"Majoity_Minority":null,"Sentiment":null},{"Totals":"Totals","M&S Dotcom Hierarchy":"M&S Dotcom","Dotcom BU":null,"Comment":"I have searched and searched for clementine gin but it is always out of stock. Very frustrating !","Feedback OSAT":"1. Very Dissatisfied","S_ecid":"4785472429669436007112741006694507216","Majoity_Minority":1.0,"Sentiment":"NEGATIVE"},{"Totals":"Totals","M&S Dotcom Hierarchy":"M&S Dotcom","Dotcom BU":null,"Comment":"Why have you got rid of the \\u2018find in store\\u2019 feature?","Feedback OSAT":"2. Fairly Dissatisfied","S_ecid":"8695593281591300898424539682589394887","Majoity_Minority":0.0,"Sentiment":"NEGATIVE"},{"Totals":"Totals","M&S Dotcom Hierarchy":"M&S Dotcom","Dotcom BU":null,"Comm

In [11]:
Final_result['Majoity_Minority'].value_counts()

1.0    127
0.0     15
Name: Majoity_Minority, dtype: int64

In [12]:
Final_result['Sentiment'].value_counts()

NEGATIVE    131
POSITIVE     11
Name: Sentiment, dtype: int64